## Import

In [1]:
from wmb import brain, cemba, mm10

from ALLCools.clustering import *
from ALLCools.mcds import MCDS
from ALLCools.plot import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
n_cell = 5
remove_lower_features = 0.2
zscore_abs_cutoff = 3
var_dim = 'chrom5k'
chrom_to_remove = ['chrX', 'chrY', 'chrM', 'chrL']

In [3]:
# Parameters
chrom_to_remove = ["chrX", "chrY", "chrM", "chrL"]
cpu = 1
group_name = "Isocortex"
mem_gb = 1
n_cell = 5
remove_lower_features = 0.2
var_dim = "chrom5k"
zscore_abs_cutoff = 3


## Select cells

In [4]:
cells = pd.read_csv('mc_cells.txt', index_col=0, header=None).index
cells.name = 'cell'

## Get adata with basic feature selection

In [5]:
mcds = MCDS.open(cemba.CEMBA_SNMC_MCDS_PATH,
                 var_dim=var_dim,
                 use_obs=cells)

mcds

<xarray.MCDS>
Dimensions:                    (cell: 94767, chrom5k: 545118)
Coordinates:
  * cell                       (cell) <U15 '10A_M_1181' ... '9F_M_2348'
  * chrom5k                    (chrom5k) <U11 'chr1_0' 'chr1_1' ... 'chrY_18348'
    chrom5k_chrom              (chrom5k) <U5 'chr1' 'chr1' ... 'chrY' 'chrY'
    chrom5k_end                (chrom5k) int64 5000 10000 ... 91740000 91744698
    chrom5k_start              (chrom5k) int64 0 5000 ... 91735000 91740000
Data variables:
    chrom5k_da_CGN-hypo-score  (cell, chrom5k) float16 dask.array<chunksize=(1000, 545118), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom5k

In [6]:
adata = mcds.get_score_adata(mc_type='CGN',
                             quant_type='hypo',
                             sparse=True,
                             binarize_cutoff=0.95,
                             loading_chunk=30000)

Loading chunk 0-30000/94767


Loading chunk 30000-60000/94767


Loading chunk 60000-90000/94767


Loading chunk 90000-94767/94767


## Basic Feature Filtering

In [7]:
remove_chromosomes(adata, exclude_chromosomes=chrom_to_remove)

492558 regions remained.


In [8]:
filter_regions(adata, n_cell=n_cell, zscore_abs_cutoff=2)

457058 regions remained.


In [9]:
remove_black_list_region(adata, black_list_path=mm10.ENCODE_BLACKLIST_PATH)

16245 features removed due to overlapping (bedtools intersect -f 0.2) with black list regions.


In [10]:
if adata.shape[1] > 50000:
    feature_sum = adata.X.sum(axis=0).A1
    use_features = feature_sum > np.quantile(feature_sum, remove_lower_features)
    adata = adata[:, use_features]

In [11]:
adata.write_h5ad('mc_input.h5ad')

In [12]:
adata

View of AnnData object with n_obs × n_vars = 94767 × 352451
    var: 'chrom', 'end', 'start'